In [5]:
!pip install requests
!pip install rcsbsearchapi


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python3.11 -m pip install --upgrade pip
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.6/177.6 kB 6.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Obtaining dependency information for tqdm from https://files.pythonhosted.org/packages/00/e5/f12a80907d0884e6dff9c16d0c0114d81b8cd07dc3ae54c5e962cc83037e/tqdm-4.66.1-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.3/78.3 kB 10.8 MB/s eta 0:00:00
  Created wheel for rcsbsearchapi: filename=rcsbsearchapi-1.4.2-py2.py3-none-any.whl size=163536 sha256=32cc6be00810f4dc3b0b93958da7b6d2fffc89eac2a1cc1d6d4965b31c5349eb
  Stored in directory: /Users/ryan/Library/Caches/pip/wheels/d5/c7/25/f3e9f96e227b7524bf0777ada1f988378f3e2e1e066f0a9634
Successfully built rcsbsearchapi

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To u

In [14]:
import re
import requests

from rcsbsearchapi.search import SequenceQuery

### Phase 1: Solicit User for UniProt ID and Query/Save the Protein's Similar Sequences

In [28]:
# Define a function that takes a Uniprot ID as input and returns True if the ID exists in Uniprot and False if it doesn't.
def is_uniprot(pid):
    uniprot_api=f"https://www.uniprot.org/uniprot/{pid}.txt"
    response = requests.get(uniprot_api)
    if response.status_code == 200:
        return True
    else:
        pid = input(f"Error: {pid} doesn't exist in Uniprot. Enter a valid Uniprot ID:") # Get UniProt ID from user
        is_uniprot(pid)

# Define a function that takes a Uniprot ID as input and returns a list similar IDs from PubChem BioAssay.
def get_fasta(pid):
    uniprot_fasta_url = f"https://www.uniprot.org/uniprot/{pid}.fasta" # Need to update summary to similarity

    response = requests.get(uniprot_fasta_url)
    if response.status_code == 200:
        # Process the response if request was successful
        fasta = response.text
        return fasta
    else:
        # Handle errors
        print("Error:", response.text)

# Prompt the user for a UniProt ID
pid = input("Enter a Uniprot ID: ")
is_uniprot(pid)
fasta = get_fasta(pid)


None


In [41]:
# Function to query PDB for sequence similar proteins to the input protein. Returns a list of similar proteins.
def get_similarpid(fasta_sequence):
    results = SequenceQuery(fasta_sequence, 1, 0.9)
    return results

sequence = re.search(r'(?<=\n)[A-Z\n]+', fasta)

fasta_sequence = sequence.group(0)
similar_pids = get_similarpid(fasta_sequence)

# results("polymer_entity") produces an iterator of IDs with return type - polymer entities
for polyid in results("polymer_entity"):
    print(polyid)

4Q21_1
8ELK_1
8ELR_1
8ELS_1
8ELT_1
8ELU_1
8ELV_1
8ELW_1
8ELX_1
8ELY_1
8ELZ_1
8EM0_1
5X9S_1
1AA9_1
1IOZ_1
1Q21_1
6Q21_1
7VV9_1
2Q21_1
6AMB_1
6KYH_2
1LFD_2
121P_1
1BKD_1
1CRP_1
1CRQ_1
1CRR_1
1CTQ_1
1GNP_1
1GNQ_1
1GNR_1
1K8R_1
1NVV_2
1NVW_1
1P2S_1
1P2T_1
1P2U_1
1P2V_1
1QRA_1
1WQ1_1
1XCM_1
1XD2_2
2RGE_1
3K8Y_1
3KUD_1
3L8Y_1
3L8Z_1
3LBH_1
3LBI_1
3LBN_1
3RRY_1
3RRZ_1
3RS0_1
3RS2_1
3RS3_1
3RS4_1
3RS5_1
3RS7_1
3RSL_1
3RSO_1
3TGP_1
4DLS_1
4DLT_1
4DLU_1
4DLW_1
4EFL_1
4G0N_1
4NYI_2
4NYJ_2
4NYM_2
4RSG_1
4URU_1
4URV_1
4URW_1
4URX_1
4URY_1
4URZ_1
4US0_1
4US1_1
4US2_1
5B2Z_1
5B30_1
5E95_2
5P21_1
5WDO_1
5WFO_1
5WFP_1
5WFQ_1
5WFR_1
6AXG_2
6BVI_3
6BVJ_1
6BVJ_3
6BVK_3
6BVL_3
6BVM_3
6CUO_3
6CUP_3
6CUR_3
6D55_3
6D56_3
6D59_3
6D5E_3
6D5G_3
6D5H_3
6D5J_3
6D5L_3
6D5M_2
6D5V_2
6D5W_2
6V94_3
6V9F_3
6V9J_3
6V9L_3
6V9M_3
6V9N_3
6V9O_3
6ZL3_1
7L0F_1
7OG9_1
7OGA_1
7OGB_1
7OGC_1
8BE6_1
8BE7_1
8BE8_1
8BE9_1
8BEA_1
8BWG_1
8CNJ_1
8CNN_1
1IAQ_1
1LF0_1
1LF5_1
1NVU_1
1NVX_1
221P_1
2LCF_1
2LWI_1
2N42_1
2N46_1
2QUZ_1
2RGB_1

### Phase 2: Query All Assays Related to all Identified Proteins from PubChem BioAssay

MDIKNSPSSLNSPSSYNCSQSILPLEHGSIYIPSSYVDSHHEYPAMTFYSPAVMNYSIPS
NVTNLEGGPGRQTTSPNVLWPTPGHLSPLVVHRQLSHLYAEPQKSPWCEARSLEHTLPVN
RETLKRKVSGNRCASPVTGPGSKRDAHFCAVCSDYASGYHYGVWSCEGCKAFFKRSIQGH
NDYICPATNQCTIDKNRRKSCQACRLRKCYEVGMVKCGSRRERCGYRLVRRQRSADEQLH
CAGKAKRSGGHAPRVRELLLDALSPEQLVLTLLEAEPPHVLISRPSAPFTEASMMMSLTK
LADKELVHMISWAKKIPGFVELSLFDQVRLLESCWMEVLMMGLMWRSIDHPGKLIFAPDL
VLDRDEGKCVEGILEIFDMLLATTSRFRELKLQHKEYLCVKAMILLNSSMYPLVTATQDA
DSSRKLAHLLNAVTDALVWVIAKSGISSQQQSMRLANLLMLLSHVRHASNKGMEHLLNMK
CKNVVPVYDLLLEMLNAHVLRGCKSSITGSECSPAEDSKSKEGSQNPQSQ



### Phase 3: Find Substances That Were Tested From Assays and Store in Dictionary

### Phase 4: Call Protein Data Bank for Pairs of InChi and UniProt Match 

### Phase 5: Output Remaining Data as a Table